# Drop outliers and interpolate

Filter out points with low confidence scores and interpolate over
missing values.


## Imports



In [ ]:
from movement import sample_data

## Load a sample dataset



In [ ]:
ds = sample_data.fetch_dataset("DLC_single-wasp.predictions.h5")
print(ds)

We see that the dataset contains the 2D pose tracks and confidence scores
for a single wasp, generated with DeepLabCut. The wasp is tracked at two
keypoints: "head" and "stinger" in a video that was recorded at 40 fps and
lasts for approximately 27 seconds.



## Visualise the pose tracks
Since the data contains only a single wasp, we use
:meth:`xarray.DataArray.squeeze` to remove
the dimension of length 1 from the data (the ``individuals`` dimension).



In [ ]:
ds.position.squeeze().plot.line(
    x="time", row="keypoints", hue="space", aspect=2, size=2.5
)

We can see that the pose tracks contain some implausible "jumps", such
as the big shift in the final second, and the "spikes" of the stinger
near the 14th second. Perhaps we can get rid of those based on the model's
reported confidence scores?



## Visualise confidence scores
The confidence scores are stored in the ``confidence`` data variable.
Since the predicted poses in this example have been generated by DeepLabCut,
the confidence scores should be likelihood values between 0 and 1.
That said, confidence scores are not standardised across pose
estimation frameworks, and their ranges can vary. Therefore,
it's always a good idea to inspect the actual confidence values in the data.

Let's first look at a histogram of the confidence scores. As before, we use
:meth:`xarray.DataArray.squeeze` to remove the ``individuals`` dimension
from the data.



In [ ]:
ds.confidence.squeeze().plot.hist(bins=20)

Based on the above histogram, we can confirm that the confidence scores
indeed range between 0 and 1, with most values closer to 1. Now let's see how
they evolve over time.



In [ ]:
ds.confidence.squeeze().plot.line(
    x="time", row="keypoints", aspect=2, size=2.5
)

Encouragingly, some of the drops in confidence scores do seem to correspond
to the implausible jumps and spikes we had seen in the position.
We can use that to our advantage.



## Filter out points with low confidence
Using the
:meth:`filter_by_confidence()\
<movement.move_accessor.MovementDataset.filtering_wrapper>`
method of the ``move`` accessor,
we can filter out points with confidence scores below a certain threshold.
The default ``threshold=0.6`` will be used when ``threshold`` is not
provided.
This method will also report the number of NaN values in the dataset before
and after the filtering operation by default (``print_report=True``).
We will use :meth:`xarray.Dataset.update` to update ``ds`` in-place
with the filtered ``position``.



In [ ]:
ds.update({"position": ds.move.filter_by_confidence()})

<div class="alert alert-info"><h4>Note</h4><p>The ``move`` accessor :meth:`filter_by_confidence()\
   <movement.move_accessor.MovementDataset.filtering_wrapper>`
   method is a convenience method that applies
   :func:`movement.filtering.filter_by_confidence`,
   which takes ``position`` and ``confidence`` as arguments.
   The equivalent function call using the
   :mod:`movement.filtering` module would be:

```python
from movement.filtering import filter_by_confidence

ds.update({"position": filter_by_confidence(position, confidence)})</p></div>
```


We can see that the filtering operation has introduced NaN values in the
``position`` data variable. Let's visualise the filtered data.



In [ ]:
ds.position.squeeze().plot.line(
    x="time", row="keypoints", hue="space", aspect=2, size=2.5
)

Here we can see that gaps (consecutive NaNs) have appeared in the
pose tracks, some of which are over the implausible jumps and spikes we had
seen earlier. Moreover, most gaps seem to be brief,
lasting < 1 second (or 40 frames).



## Interpolate over missing values
Using the
:meth:`interpolate_over_time()\
<movement.move_accessor.MovementDataset.filtering_wrapper>`
method of the ``move`` accessor,
we can interpolate over the gaps we've introduced in the pose tracks.
Here we use the default linear interpolation method (``method=linear``)
and interpolate over gaps of 40 frames or less (``max_gap=40``).
The default ``max_gap=None`` would interpolate over all gaps, regardless of
their length, but this should be used with caution as it can introduce
spurious data. The ``print_report`` argument acts as described above.



In [ ]:
ds.update({"position": ds.move.interpolate_over_time(max_gap=40)})

<div class="alert alert-info"><h4>Note</h4><p>The ``move`` accessor :meth:`interpolate_over_time()\
   <movement.move_accessor.MovementDataset.filtering_wrapper>`
   is also a convenience method that applies
   :func:`movement.filtering.interpolate_over_time`
   to the ``position`` data variable.
   The equivalent function call using the
   :mod:`movement.filtering` module would be:

```python
from movement.filtering import interpolate_over_time

ds.update({"position": interpolate_over_time(
    position_filtered, max_gap=40
)})</p></div>
```


We see that all NaN values have disappeared, meaning that all gaps were
indeed shorter than 40 frames.
Let's visualise the interpolated pose tracks.



In [ ]:
ds.position.squeeze().plot.line(
    x="time", row="keypoints", hue="space", aspect=2, size=2.5
)

## Log of processing steps
So, far we've processed the pose tracks first by filtering out points with
low confidence scores, and then by interpolating over missing values.
The order of these operations and the parameters with which they were
performed are saved in the ``log`` attribute of the ``position`` data array.
This is useful for keeping track of the processing steps that have been
applied to the data. Let's inspect the log entries.



In [ ]:
for log_entry in ds.position.log:
    print(log_entry)

## Filtering multiple data variables
All :mod:`movement.filtering` functions are available via the
``move`` accessor. These ``move`` accessor methods operate on the
``position`` data variable in the dataset ``ds`` by default.
There is also an additional argument ``data_vars`` that allows us to
specify which data variables in ``ds`` to filter.
When multiple data variable names are specified in ``data_vars``,
the method will return a dictionary with the data variable names as keys
and the filtered DataArrays as values, otherwise it will return a single
DataArray that is the filtered data.
This is useful when we want to apply the same filtering operation to
multiple data variables in ``ds`` at the same time.

For instance, to filter both ``position`` and ``velocity`` data variables
in ``ds``, based on the confidence scores, we can specify
``data_vars=["position", "velocity"]`` in the method call.
As the filtered data variables are returned as a dictionary, we can once
again use :meth:`xarray.Dataset.update` to update ``ds`` in-place
with the filtered data variables.



In [ ]:
ds["velocity"] = ds.move.compute_velocity()
filtered_data_dict = ds.move.filter_by_confidence(
    data_vars=["position", "velocity"]
)
ds.update(filtered_data_dict)